In [1]:
import requests
from bs4 import BeautifulSoup
from abc import ABC
import re

In [2]:
class Site:
    def __init__(self,name,posts):
        self.name = name
        self.posts = posts
        
    def __repr__(self):
        #stub
        return f'{self.name}\n\n\n{self.posts_to_str()}'
    
    def filter_posts(self):
        '''
        remove unecessary posts from the posts attribute.
        
        to be implemented by a subclass.'''
        pass
    
    def posts_to_str(self):
        return '\n\n\n'.join([f'{post}' for post in self.posts])

In [3]:
class Post:
    def __init__(self,title,content,link):
        self.title = title
        self.content = content
        self.link = link
    
    def title_contains(self, target):
        return target in self.title.lower()
    
    def __repr__(self):
        #stub
        return f'TITLE:\n\n{self.title}\n\nCONTENT:\n\n{self.content}\n\nLINK:\n\n{self.link}'

In [4]:
class Source():
    def __init__(self,name,url):
        self.name = name
        self.url = url
        
    def download(self) -> Site:
        '''returns Site'''
        source_html = requests.get(self.url).text
        soup = BeautifulSoup(source_html,'html.parser')
        
        site = self.parse(soup)
        site.filter_posts()
        
        return site
    
    def parse(self,soup) -> Site:
        '''to be implemented by a subclass'''
        pass

In [5]:
class MacrumorsSite(Site):
    def filter_posts(self):
        self.remove_giveaways_posts()
        self.remove_deals()
        
    def remove_deals(self):
        self.remove_posts(lambda post: post.title_contains('deals:'))
        
    def remove_giveaways_posts(self):
        self.remove_posts(lambda post: post.title_contains('giveaway'))
    
    def remove_posts(self, filter_fn):
        '''filter_fn is a function that returns true
        if the post should be removed.'''
        
        self.posts = list(filter(lambda post: not filter_fn(post), self.posts))

In [6]:
class MacrumorsSource(Source):
    def parse(self,soup) -> MacrumorsSite:
        contents = self.parse_content(soup)
        titles = self.parse_titles(soup)
        links = self.parse_links(soup)
        
        posts_raw = zip(titles,contents,links)
        posts = [Post(*post_data) for post_data in posts_raw]
        
        return MacrumorsSite(self.name, posts)
    
    def parse_content(self,soup):
        content_list = soup.find_all(class_='js-contentInner')
        content_list = [self.cleanup_content(content.get_text()) for content in content_list]
        return content_list
    
    def parse_titles(self,soup):
        articles = soup.find_all(class_='js-article')
        titles = map(self.get_title, articles)
        return [title.get_text() for title in titles]
    
    def parse_links(self,soup):
        articles = soup.find_all(class_='js-article')
        titles = map(self.get_title, articles)
        links = [title.find('a')['href'] for title in titles]
        return links
    
    def get_title(self,article):
        '''
        gets the first child of the article.
        
        This is more resilient than looking for certain tags.
        '''
        return next(article.children)
        
    def cleanup_content(self,content):
        # corrects the content text.
        replacements = [
            ('\r ','\n'*2), #replace linebreaks
            ('  ', ' '), #fix double spaces
            ('\n','\n\n')
        ]
        for r in replacements:
            content = content.replace(*r)
            
        # removes repertitive whitespace
        content = re.sub(r'\n\n+[\n]','\n\n',content)

        # remove errors in the content text
        errors = [
            '\u200c',
            'img.lazyload { display: none; }',
            'img.lazyload { display: none; } '
        ]
        
        for e in errors:
            # removes error
            content = content.replace(e, '')
            
        # remove leading blankspace
        paragraphs = content.split('\n'*2)
        paragraphs = [p.strip() for p in paragraphs]
        content = ('\n'*2).join(paragraphs)
        
        # remove trailing blankspace
        content = content.strip()
            
        return content

In [7]:
def prevent_exit():
    input('Press enter to exit.')

In [8]:
m = MacrumorsSource('Macrumors','https://www.macrumors.com/')
site = m.download()

In [ ]:
print(site)
prevent_exit()

Macrumors


TITLE:

U.S. ITC Launches Investigation Into Capacitive Devices Made by Apple and Others Following Patent Infringement Complaint

CONTENT:

The United States International Trade Commission today announced an investigation into possible patent violations involving capacitive touch-controlled devices, computers, and components created by Apple, Amazon, Samsung, and others, reports Reuters.

According to a document announcing the investigation [PDF], it stems from a February complaint filed by Irish company Neodron claiming Apple and other companies are infringing on Neodron-owned patents related to touch-based devices.

The investigation is based on a complaint filed by Neodron, Ltd., of Dublin, Ireland, on February 14, 2020. The complaint alleges violations of section 337 of the Tariff Act of 1930 in the importation into the United States and sale of certain capacitive touch-controlled mobile devices, computers, and components thereof that infringe patents asserted by the co